In [1]:
import csv
import itertools
import math
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib #引入画图数据库
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import scipy.fftpack as fftpack
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from matplotlib.dates import MonthLocator, DateFormatter
from sklearn.ensemble import RandomForestRegressor
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.tsa.stattools import adfuller
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.stats.stattools import durbin_watson  # DW检验
from statsmodels.graphics.api import qqplot  # qq图
from scipy import io, integrate, linalg, signal
from scipy.sparse.linalg import eigs
from scipy.integrate import odeint, quad

from statsmodels.tsa.stattools import acf
from scipy.optimize import minimize
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import ConstantKernel, RBF

/Users/bulingzheng/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [25]:
def transferdata_try():
    # 读取CSV文件
    # df = pd.read_csv("备品备件库存移动清单查询.csv", encoding="utf-8")
    df = pd.read_csv("2018-2023年备件领用记录.csv", encoding="utf-8")
    # df = pd.read_csv("杭州领用记录.csv", encoding="utf-8")
    # df = pd.read_csv("宁波领用记录.csv", encoding="utf-8")
    # 将日期列转换为日期时间格式
    df['凭证日期'] = pd.to_datetime(df['凭证日期'], format='%Y%m%d')
    # df['凭证日期'] = pd.to_datetime(df['凭证日期'], format='%Y/%m/%d')
    # df['凭证日期'] = pd.to_datetime(df['凭证日期'], format='%Y-%m-%d')
    # 提取年份和月份列
    df['年份'] = df['凭证日期'].dt.year
    df['月份'] = df['凭证日期'].dt.month
    # print(df)
    # # 提取领用科室中含有"杭州"的行
    # hangzhou_df = df[df['领用科室'].str.contains('杭州')]
    # # 提取领用科室中含有"宁波"的行
    # ningbo_df = df[df['领用科室'].str.contains('宁波')]
    # # 打印结果
    # print("杭州领用记录：")
    # print(hangzhou_df)
    # print("\n宁波领用记录：")
    # print(ningbo_df)
    # hangzhou_df.to_csv("杭州领用记录.csv", index=False, encoding='utf-8')
    # ningbo_df.to_csv("宁波领用记录.csv", index=False, encoding='utf-8')
    # 使用pivot_table函数将数据透视为所需的格式
    df_pivot = df.pivot_table(index='物料号', columns=['年份', '月份'], values='数量', aggfunc='sum', fill_value=0)
    # # 获取物料号和账册编号的对应关系，筛选帐册编号为[C]的记录，并且只取每个物料号的第一个帐册编号
    # material_to_A = df[df['帐册编号'] == '[A]'][['物料号', '帐册编号']].drop_duplicates(
    #     subset=['物料号']).reset_index(drop=True)
    # material_to_B = df[df['帐册编号'] == '[B]'][['物料号', '帐册编号']].drop_duplicates(
    #     subset=['物料号']).reset_index(drop=True)
    # material_to_C = df[df['帐册编号'] == '[C]'][['物料号', '帐册编号']].drop_duplicates(
    #     subset=['物料号']).reset_index(drop=True)
    #
    #
    # print(material_to_A)
    # print(material_to_B)
    # print(material_to_C)
    # # 将筛选后的数据保存到 CSV 文件中
    # material_to_C.to_csv('帐册C数据_all.csv', index=False, encoding='utf-8')

    # 重置列索引，将年份和月份组合为列名
    df_pivot.columns = [f"{year}.{month:02}" for (year, month) in df_pivot.columns]

    # 将列名转换为DateTime对象
    df_pivot.columns = pd.to_datetime(df_pivot.columns, format='%Y.%m')

    # 按季度对数据进行汇总
    df_quarters = df_pivot.resample('Q', axis=1).sum()
    # 为每个季度列重命名
    quarter_names = ['第1季度', '第2季度', '第3季度', '第4季度']

    # 获取数据中实际存在的年份
    years = df_quarters.columns.year.unique()
    # 生成对应的季度名称
    new_column_names = []
    for year in years:
        quarters_in_year = 3 if year == max(years) else 4  # 最后一年的季度数量可能不足四个
        new_column_names.extend([f"{year} {quarter_names[i]}" for i in range(quarters_in_year)])


    df_quarters.columns = new_column_names
    # 在这里先删去2023第三季度，后续在process里就不删了
    # 删除名为 "2023 第3季度" 的列
    df_quarters = df_quarters.drop(columns=['2023 第3季度'])

    # print("zheshi1:",df_quarters.columns)
    non_zero_demand_count = (df_quarters.iloc[:, 0:] != 0).sum(axis=1)
    print('zhehi',non_zero_demand_count)
    # print(df_pivot)
    # 计算ADI并添加到最后一列
    # 计算非零需求的总数
    # non_zero_demand_count = (df_pivot.iloc[:, 0:] != 0).sum(axis=1)

    # 计算每年的年度总和
    for year in range(df['年份'].min(), df['年份'].max() + 1):
        if year == 2023:
            end_quarter = 2  # 2023年只有2个季度
        else:
            end_quarter = 4

        columns_for_year = []
        for quarter in range(1, end_quarter + 1):
            columns_for_year.append(f"{year} 第{quarter}季度")

        year_sum = df_quarters[columns_for_year].sum(axis=1)
        year_mean = df_quarters[columns_for_year].mean(axis=1)
        year_std = df_quarters[columns_for_year].std(axis=1)
        year_by = year_std / year_mean

        # 在每年末尾插入年度总和列
        insert_position = df_quarters.columns.get_loc(f"{year} 第{end_quarter}季度") + 1
        df_quarters.insert(insert_position, f"{year}年度总和", year_sum)
        df_quarters.insert(insert_position + 1, f"{year}年均值", year_mean)
        df_quarters.insert(insert_position + 2, f"{year}年标准差", year_std)
        df_quarters.insert(insert_position + 3, f"{year}年变异系数", year_by)
    # print(df_pivot)
    exclude_cols = ['年度总和', '年均值', '年标准差', '年变异系数']
    # filtered_cols = [col for col in df_pivot.columns if all(exclude not in col for exclude in exclude_cols)]
    filtered_cols = [str(col) for col in df_quarters.columns if
                     all(exclude not in str(col) for exclude in exclude_cols)]

    all_years_mean = df_quarters[filtered_cols].mean(axis=1)
    # print(all_years_mean)
    all_years_var = df_quarters[filtered_cols].var(axis=1)
    all_years_std = df_quarters[filtered_cols].std(axis=1)
    # 计算所有年份的均值（不包括年度总和列）
    # all_years_mean = df_quarters[[col for col in df_quarters.columns if '年度总和' not in col]].mean(axis=1)
    # all_years_var = df_quarters[[col for col in df_quarters.columns if '年度总和' not in col]].var(axis=1)
    # all_years_std = df_quarters[[col for col in df_quarters.columns if '年度总和' not in col]].std(axis=1)
    df_quarters['所有年均值'] = all_years_mean
    df_quarters['所有方差'] = all_years_var
    df_quarters['所有标准差'] = all_years_std
    df_quarters['所有年变异系数'] = (all_years_std / all_years_mean)
    df_pivot = df_quarters
    # 计算ADI并添加到最后一列
    # 计算非零需求的总数

    df_pivot['ADI'] = 22 / non_zero_demand_count

    # 重置索引，使物料号成为列
    df_pivot.reset_index(inplace=True)

    # 添加物料描述列
    material_description = df[['物料号', '物料描述']].drop_duplicates(subset=['物料号']).set_index('物料号')
    df_pivot = df_pivot.join(material_description, on='物料号')
    # print(len(df_pivot))
    # df_pivot.to_csv("all数据表.csv", index=False, encoding='utf-8')
    # 找出所有年均值为0的物料数量
    zero_mean_count = (df_pivot['所有年均值'] == 0).sum()
    print(f"所有年均值为0的物料数量：{zero_mean_count}")

    # 创建"间歇性数据"子数据集
    intermittent_data = df_pivot[(df_pivot['ADI'] > 1.32) & (abs(df_pivot['所有年变异系数']) <= 0.7) &
    (df_pivot['所有年均值'] < 0)]

    # 创建"块状需求"子数据集
    block_demand_data = df_pivot[(df_pivot['ADI'] > 1.32) & (abs(df_pivot['所有年变异系数']) > 0.7) &
    (df_pivot['所有年均值'] < 0)]
    # 创建"需求平稳"子数据集
    stable_demand_data = df_pivot[(df_pivot['ADI'] <= 1.32) & (abs(df_pivot['所有年变异系数']) <= 0.7) &
    (df_pivot['所有年均值'] < 0)]
    # 创建"不稳定需求"子数据集
    unstable_demand_data = df_pivot[(df_pivot['ADI'] <= 1.32) & (abs(df_pivot['所有年变异系数']) > 0.7) &
    (df_pivot['所有年均值'] < 0)]
    #
    # 打印"间歇性数据"子数据集
    print("间歇性数据 (ADI > 1.32 且 CV² <= 0.49):", "数量:", len(intermittent_data))
    print(intermittent_data)

    # 打印"块状需求"子数据集
    print("块状需求 (ADI > 1.32 且 CV² > 0.49):", "数量:", len(block_demand_data))
    print(block_demand_data)
    #
    # 打印"需求平稳"子数据集
    print("需求平稳 (ADI <= 1.32 且 CV² <= 0.49):", "数量:", len(stable_demand_data))
    print(stable_demand_data)
    stable_demand_data.to_csv("平稳杭宁.csv", index=False, encoding='utf-8')
    # 提取稳定需求数据中的物料号
    material_numbers_stable = stable_demand_data['物料号']
    # # 使用 merge 函数，将两个数据框根据物料号进行合并，判断是否在 df_c 中
    merged_df = pd.merge(material_numbers_stable, material_to_C[['物料号']], on='物料号', how='inner')
    print(len(merged_df))
    merged_df.to_csv('都包含帐册C数据.csv', index=False, encoding='utf-8')
    # 提取需求平稳数据集中 ADI 等于 1 的数据
    adi_equals_1_data = stable_demand_data[stable_demand_data['ADI'] == 1]
    # # 打印 ADI 等于 1 的数据
    print("需求平稳且 ADI = 1 的数据:", "数量:", len(adi_equals_1_data))
    print(adi_equals_1_data)
    adi_equals_1_data.to_csv("稳定杭宁.csv", index=False, encoding='utf-8')

    # 打印"不稳定需求"子数据集
    print("不稳定需求 (ADI <= 1.32 且 CV² > 0.49):", "数量:", len(unstable_demand_data))
    print(unstable_demand_data)
    # unstable_demand_data.to_csv("不平稳杭宁.csv", index=False, encoding='utf-8')
    # 提取需求不平稳数据集中 ADI 等于 1 的数据
    adi_equals_1_data_2 = unstable_demand_data[unstable_demand_data['ADI'] == 1]
    # # 打印 ADI 等于 1 的数据
    print("不稳定需求且 ADI = 1 的数据:", "数量:", len(adi_equals_1_data_2))
    print(adi_equals_1_data_2)
    # adi_equals_1_data_2.to_csv("不稳定杭宁.csv", index=False, encoding='utf-8')

    intermittent_data.to_csv("间歇性需求_季度_all.csv", index=False, encoding='utf-8')
    block_demand_data.to_csv("块状需求_季度_all.csv", index=False, encoding='utf-8')
    unstable_demand_data.to_csv("不稳定需求_季度_all.csv", index=False, encoding='utf-8')
    stable_demand_data.to_csv("需求平稳_季度_杭州.csv", index=False, encoding='utf-8')
    unstable_demand_data.to_csv("不稳定需求_季度_杭州.csv", index=False, encoding='utf-8')
    print(df_pivot)
    df_pivot.to_csv("季度数据_all.csv", index=False, encoding='utf-8')


In [26]:
def preprocess_try():
    # 读取CSV文件
    df = pd.read_csv("不稳定杭宁.csv", encoding="utf-8")

    #df = pd.read_csv("新不稳定需求_季度_ADI=1_hz.csv", encoding="ANSI")
    #df = pd.read_csv("新需求平稳_季度_ADI=1_nb.csv", encoding="ANSI")
    # df = pd.read_csv("新不稳定需求_季度_ADI=1_nb.csv", encoding="ANSI")

    # print(df)
    # 定义一个函数，用于将负数转换为正数
    def make_positive(x):
        if isinstance(x, (int, float)):
            return abs(x)
        else:
            return x  # 保持其他类型不变

    # 使用applymap方法应用函数，只影响数值列
    df = df.applymap(lambda x: make_positive(x))

    # 获取要删除的列名列表
    # columns_to_delete = [col for col in df.columns if col.endswith(('均值', '标准差', '方差', '变异系数', '总和', 'ADI'))]
    columns_to_delete = [col for col in df.columns if col.endswith(('均值', '标准差', '方差', '变异系数', '总和', 'ADI','物料描述'))]


    # 使用 drop 方法删除这些列
    df.drop(columns=columns_to_delete, inplace=True)

    # 更改日期格式
    df.columns = [col.replace('.', '/') for col in df.columns]

    # 获取列索引
    columns = df.columns

    # 重置列索引为默认整数索引
    df.reset_index(drop=True, inplace=True)

    # 创建一个新行，将列索引作为值
    new_row = pd.DataFrame([columns], columns=df.columns)

    # 将新行添加到DataFrame的顶部
    df = pd.concat([new_row, df], axis=0, ignore_index=True)

    # 转置矩阵
    df = df.set_index(df.columns[0]).transpose()

    # 重置行索引为默认整数索引
    df.reset_index(drop=True, inplace=True)

    # 删除行索引的列名
    df = df.rename_axis(None, axis=1)

    # 更换'物料号'列索引为'time'
    df = df.rename(columns={'物料号': 'time'})

    # # 将'time'列转换为时间格式
    # df['time'] = pd.to_datetime(df['time'], format='%Y/%m')
    # print(df)
    # 删去存在0的列
    # non_zero_columns = df.loc[:, (df != 0).all()]
    # df = non_zero_columns

    # print(df)
    # df.to_csv("宁波厂全部备件季度数据.csv", index=False, encoding='ANSI')
    return df

In [27]:
# 简单移动平均法预测
def SMAforecast_try(n):
    # 参数
    # n = 6               # 读取的列索引序号
    window_size = 3  # 窗口期

    # 数据预处理
    df = preprocess_try()
    # print(df.columns)
    # 读取列索引名
    col = df.columns

    # print(df.iloc[:,n])
    # # 计算移动平均值
    y = df.iloc[:, n].rolling(window=window_size).mean()
    # y_2 = df.iloc[:, n].ewm(span=window_size).mean()
    # 使用rolling窗口并应用自定义权重

    # # 定义自定义权重
    # custom_weights = [0.1, 0.3, 0.6]
    # y_2 = df.iloc[:, n].rolling(window=window_size).apply(
    #     lambda x: (x * custom_weights).sum())

    # 定义一组待尝试的权重组合
    weight_combinations = [
        [0.1, 0.3, 0.6],
        [0.2, 0.3, 0.5],
        [0.2, 0.4, 0.4],
        [0.3, 0.3, 0.4],
        [0.1, 0.4, 0.5],
        [0.1, 0.2, 0.7],
        # 可以添加更多的组合
    ]

    mape_wight = float('inf')  # 初始化最佳 MAPE 为无穷大
    best_weights = None  # 初始化最佳权重组合为空
    y_2_predict = None
    for weights in weight_combinations:
        # 使用当前权重组合计算移动加权平均
        y_2 = df.iloc[:, n].rolling(window=window_size).apply(
            lambda x: (x * weights).sum())
        # 计算 MAPE
        mape_wight_2 = np.mean(np.abs((y_2[2:-1].values - df.iloc[3:, n].values) / df.iloc[3:, n].values))
        # 更新最佳 MAPE 和对应的权重组合
        if mape_wight_2 < mape_wight:
            mape_wight = mape_wight_2
            best_weights = weights
            y_2_predict =y_2
    print("最佳权重组合:", best_weights)
    print("最佳 MAPE:", mape_wight)
    print("最佳 加权预测值:", y_2_predict)

    # y.fillna(method='ffill', inplace=True)  # 前向填充
    # y.fillna(method='bfill', inplace=True)  # 后向填充
    # y_2_predict.fillna(method='ffill', inplace=True)  # 前向填充
    # y_2_predict.fillna(method='bfill', inplace=True)  # 后向填充
    # # print(y, y_2, df)

    # MAPE
    mape_single = np.mean(np.abs((y[2:-1].values - df.iloc[3:, n].values) / df.iloc[3:, n].values))
    # mape_wight = np.mean(np.abs((y_2[2:-1].values - df.iloc[3:, n].values) / df.iloc[3:, n].values))
    # mae_single = mean_absolute_error(df.iloc[3:, n].values, y[2:-1].values)
    # mae_wight = mean_absolute_error(df.iloc[3:, n].values, y_2[2:-1].values)

    # print('预测值', y[2:-1].values, '加权',y_2_predict[2:-1].values)
    # print('真实值', df.iloc[3:, n].values)
    # print(y[2:-1].values - df.iloc[3:, n].values)
    print('mape', mape_single)
    print('mape加权', mape_wight)

    # print(df['time'])

    time_values = df['time'].values
    plt.figure(figsize=(10, 6))
    plt.plot(df['time'], df.iloc[:, n], label='原始数据',color='black')
    plt.plot(time_values[1:], y[:-1], label=f'一次 ({window_size}天)', color='b')
    plt.plot(time_values[1:], y_2_predict[:-1], label=f'加权 ({window_size}天)', color='r')
    plt.xlabel('日期')
    plt.ylabel('数值')
    # plt.title(f'物料号：{col[n]}\n一次移动平均 MAPE:{round(mape_single, 2)} \n 加权移动平均法 MAPE:{round(mape_wight,2)}')
    plt.title(f'物料号：{col[n]}\n一次移动平均 MAPE:{round(mape_single, 2)}  \n 加权移动平均法 MAPE:{round(mape_wight, 2)} ')
    plt.legend()
    plt.xticks(rotation=45)
    plt.grid(True,linestyle='--', linewidth=0.5)
    plt.tight_layout()
    plt.savefig('SMA')
    plt.show()

    return mape_single, mape_wight, col
def SMA_run():
    # 创建一个空列表，用于存储 MAPE 值
    sma_mape_values = []
    for n in data_SMA:
        # 运行 SMAforecast() 并计算 MAPE
        mape_single, mape_wight, col = SMAforecast_try(n)
        # 取出对应的物料号
        material_number = col[n]
        # 将 MAPE 值作为元组添加到列表中
        sma_mape_values.append((col[n], mape_single, mape_wight))
    # 将 MAPE 值写入文件
    with open("SMA不平稳杭宁.txt", "w") as f:
        # 循环遍历每个 n 对应的 MAPE 值，并将其写入文件
        for material_number, mape_single, mape_wight in sma_mape_values:
            f.write(
                f"物料号：{material_number}, MAPE Single: {round(mape_single, 2)}, MAPE Weight: {round(mape_wight, 2)}\n")
    csv_filename = "SMA不平稳杭宁.csv"
    with open(csv_filename, "a", newline='') as f:  # 使用追加模式，并且指定 newline='' 来避免空行
        writer = csv.writer(f)
        # 循环遍历每个 n 对应的 MAPE 值，并将其写入文件
        for material_number, mape_single, mape_wight in sma_mape_values:
            writer.writerow([material_number, round(mape_single, 2), round(mape_wight, 2)])
        plt.show()
    return
if __name__ == '__main__':
    # 物料取值
    data_SMA = range(1,2)
    # 运行
    SMA_run()

FileNotFoundError: [Errno 2] No such file or directory: '不稳定杭宁.csv'

In [20]:
# 指数平滑预测(一、二、三)
def EMAforecast(n):
    # 数据预处理
    df = preprocess_try()
    # 读取列索引名
    col = df.columns
    # 转换为日期时间索引
    df.set_index('time', inplace=True)
    # 划分训练和测试
    train = df[:'2022 第2季度']
    test = df['2022 第3季度':]
    train = train.iloc[:, n - 1]
    train = train.astype(float)
    test = test.iloc[:, n - 1]
    test = test.astype(float)
    # 定义一个指数平滑模型的超参数优化函数
    # 遍历不同的 alpha 值，通过拟合一次指数平滑模型到训练数据，并在测试集上计算 MAE，找到最佳的 alpha 值，
    # 然后返回最佳的 alpha 值和对应的 MAE 值。这样可以帮助选择在指数平滑模型中的最佳超参数，以获得最佳的预测性能。
    def ses_optimizer(train, alphas, step=4):  # step for length of test set
        # 初始化最佳的 alpha 值和最佳的 MAPE 为无穷大
        best_alpha, best_mape_one, best_mae_one = None, float("inf"), float("inf")
        # 遍历 alpha 值的候选列表
        for alpha in alphas:
            # 使用当前的 alpha 值拟合一次指数平滑模型到训练数据
            ses_model = SimpleExpSmoothing(train).fit(smoothing_level=alpha)
            # 使用拟合的模型预测未来 step 个时间点的值
            y_pred = ses_model.forecast(step)
            # 计算预测值与测试集的平均绝对误差（MAPE）
            mape_one = np.mean(np.abs((y_pred.values - test.values) / test.values))
            # 如果当前 alpha 值的 MAPE小于最佳 MAPE，则更新最佳 alpha 值和最佳 MAPE
            if mape_one < best_mape_one:
                best_alpha, best_mape_one = alpha, mape_one
            # 打印当前 alpha 值和对应的 MAPE 值
            print("alpha:", round(alpha, 2), "mape:", round(mape_one, 4))
        # 打印最佳的 alpha 值和对应的最佳 MAPE 值
        print("best_alpha:", round(best_alpha, 2), "best_mape_one", round(best_mape_one, 4))
        # 返回最佳的 alpha 值和最佳的 MAPE值
        return best_alpha, best_mape_one

    # 定义一个二次指数平滑模型的超参数优化函数
    # 用于选择最佳的 alpha 和 beta 参数
    def des_optimizer(train, alphas, betas, step=4):
        best_alpha, best_beta, best_mape_two, best_mae_two = None, None, float("inf"), float("inf")  # 初始化最佳参数和最佳MAPE
        for alpha in alphas:  # 遍历alpha参数列表
            for beta in betas:  # 遍历beta参数列表
                # 使用ExponentialSmoothing模型拟合训练数据，设置趋势为"add"，并指定alpha和beta参数
                des_model = ExponentialSmoothing(train, trend="add").fit(smoothing_level=alpha, smoothing_slope=beta)
                y_pred = des_model.forecast(step)  # 预测未来step个时间步长的值
                # 计算预测值与测试集的平均绝对误差（MAPE）
                mape_two = np.mean(np.abs((y_pred.values - test.values) / test.values))
                if mape_two < best_mape_two:  # 如果当前MAPE更好（更小），则更新最佳参数和最佳MAPE
                    best_alpha, best_beta, best_mape_two = alpha, beta, mape_two
                # 打印当前参数组合的结果
                print("alpha:", round(alpha, 2), "beta:", round(beta, 2), "mape:", round(mape_two, 4))
        # 打印最佳参数和对应的最佳MAE
        print("best_alpha:", round(best_alpha, 2), "best_beta:", round(best_beta, 2), "best_mape:",
              round(best_mape_two, 4))
        return best_alpha, best_beta, best_mape_two # 返回最佳参数和最佳MAPE

    # 定义一个三次指数平滑模型的超参数优化函数
    # 用于选择最佳的 alpha、beta、gamma 参数
    def tes_optimizer(train, abg, step=4):
        # 初始化最佳的 alpha、beta、gamma 和最佳 MAPE
        best_alpha, best_beta, best_gamma, best_mape_three, best_season_periods = None, None, None, float("inf"), float("inf")
        # 遍历参数组合列表
        for comb in abg:
            # 使用给定的参数组合构建三次指数平滑模型-加法模型
            tes_model = ExponentialSmoothing(train, trend="add", seasonal="add", seasonal_periods=comb[3]). \
                fit(smoothing_level=comb[0], smoothing_slope=comb[1], smoothing_seasonal=comb[2])
            # 使用模型预测未来的时间点
            y_pred = tes_model.forecast(step)
            mape_three = np.mean(np.abs((y_pred.values - test.values) / test.values))

            # 如果当前 MAPE 更好（更小），则更新最佳参数和最佳 MAPE
            if mape_three < best_mape_three:
                best_alpha, best_beta, best_gamma, best_season_periods, best_mape_three = comb[0], comb[1], comb[2],comb[3] ,mape_three
            # 打印当前参数组合的结果
            print("alpha:", round(comb[0], 2), "beta:", round(comb[1], 2), "gamma:", round(comb[2], 2), "season:",round(comb[3],2),"mape:",
                  round(mape_three, 2))
        # 打印最佳参数和对应的最佳 MAE
        print("best_alpha:", round(best_alpha, 2), "best_beta:", round(best_beta, 2), "best_gamma:",
              round(best_gamma, 2),"best_season:",round(best_season_periods),
              "best_mape:", round(best_mape_three, 4))
        # 返回最佳的 alpha、beta、gamma 和最佳 MAPE
        return best_alpha, best_beta, best_gamma, best_season_periods, best_mape_three

    # 一次指数；包含了一系列从0.1到1.0的数字，间隔为0.1（更精确可选间隔为0.01）
    alphas = np.arange(0.10, 1, 0.10)
    # 二次指数
    alphas_d = np.arange(0.10, 1, 0.10)
    betas_d = np.arange(0.10, 1, 0.10)
    # 三次指数   保存最优的参数值，寻找规律，缩小范围，寻找对应参数的普适性；找找最优参数的规律，比如：某个区间
    alphas_t = betas_t = gammas_t = np.arange(0.10, 1, 0.10)
    # 季节性周期
    season_t = np.arange(2, 7, 1)
    abg = list(itertools.product(alphas_t, betas_t, gammas_t, season_t))

    # 保存值
    best_alpha, best_mape_one = ses_optimizer(train, alphas)
    best_alpha_d, best_beta_d, best_mape_two = des_optimizer(train, alphas_d, betas_d)
    best_alpha_t, best_beta_t, best_gamma_t,best_season_periods, best_mape_three = tes_optimizer(train, abg)

    # 一次指数平滑
    ses_model = SimpleExpSmoothing(train).fit(smoothing_level=best_alpha)
    # 二次指数平滑
    des_model = ExponentialSmoothing(train, trend="add").fit(smoothing_level=best_alpha_d, smoothing_slope=best_beta_d)
    # 三次指数平滑Holt-Winters
    tes_model = ExponentialSmoothing(train, trend="add", seasonal="add", seasonal_periods=best_season_periods). \
        fit(smoothing_level=best_alpha_t, smoothing_trend=best_beta_t, smoothing_seasonal=best_gamma_t)

    # 4个预测值
    y_pred = ses_model.forecast(4)
    y_pred_d = des_model.forecast(4)
    y_pred_t = tes_model.forecast(4)
    print("真实值：",test.values)
    print("三次预测值：",y_pred_t)

    # 作图
    def plot_forecast(train, test, y_pred, title):
        mape = np.mean(np.abs((y_pred.values - test.values) / test.values))
        plt.figure(figsize=(10, 6))
        # 合并训练和测试数据
        full_data = pd.concat([train, test])
        plt.plot(full_data.index, full_data.values, label=' Training Data', color='black')  # 合并后的数据
        plt.plot(test.index, y_pred, label='Forecast', color='red')
        plt.title(f"{df.columns[n - 1]}, {title}, MAPE:{round(mape, 2)}")
        plt.xlabel('Time')
        plt.ylabel('Values')
        plt.xticks(rotation=45)
        plt.grid(True, linestyle='--', linewidth=0.5)
        plt.legend()
        plt.tight_layout()
        plt.savefig(title)
        plt.show()

    plot_forecast(train, test, y_pred, "Single Exponential Smoothing")
    plot_forecast(train, test, y_pred_d, "Double Exponential Smoothing")
    plot_forecast(train, test, y_pred_t, "Triple Exponential Smoothing")
    return best_mape_one, best_mape_two, best_mape_three, col, best_alpha_d, best_beta_d, best_alpha_t, best_beta_t, best_gamma_t, best_season_periods

# 运行指数平滑法、写入
def EMA_run():
    # 创建一个空列表，用于存储 MAPE 值
    ema_mape_values = []
    for n in data_EMA:
        # 运行 SMAforecast() 并计算 MAPE
        best_mape_one, best_mape_two, best_mape_three, col, best_alpha_d, best_beta_d, best_alpha_t, best_beta_t, best_gamma_t, best_season_periods = EMAforecast(n)
        # 取出对应的物料号
        material_number = col[n]
        print("这是", material_number)
        print(best_mape_one, best_mape_two, best_mape_three, best_alpha_d, best_beta_d, best_alpha_t, best_beta_t, best_gamma_t, best_season_periods)
        # 将 MAPE 值作为元组添加到列表中
        ema_mape_values.append((material_number, best_mape_one, best_mape_two, best_mape_three, best_alpha_d, best_beta_d, best_alpha_t, best_beta_t, best_gamma_t, best_season_periods))
    # # 将 MAPE 值写入文件
    # with open("EMA平稳杭宁_4.txt", "w") as f:
    #     # 循环遍历每个 n 对应的 MAPE 值，并将其写入文件
    #     for material_number, best_mapex_one, best_mapex_two, best_mapex_three, best_alpha_d, best_beta_d, best_alpha_t, best_beta_t, best_gamma_t, best_season_periods in ema_mape_values:
    #         f.write(
    #             f"物料号：{material_number}, MAPEX One: {round(best_mapex_one, 2)}, MAPE Two: {round(best_mapex_two, 2)},MAPE Three:{round(best_mapex_three, 2)},TWO alpha beta:{round(best_alpha_d, 2), round(best_beta_d, 2)},Three alpha beta gamma:{round(best_alpha_t, 2), round(best_beta_t, 2), round(best_gamma_t, 2)},season_prriods:{best_season_periods}\n")
    # csv_filename = "EMA平稳杭宁_4.csv"
    # with open(csv_filename, "a", newline='') as f:  # 使用追加模式，并且指定 newline='' 来避免空行
    #     writer = csv.writer(f)
    #     # 循环遍历每个 n 对应的 MAPE 值，并将其写入文件
    #     for material_number, best_mapex_one, best_mapex_two, best_mapex_three, best_alpha_d, best_beta_d, best_alpha_t, best_beta_t, best_gamma_t, best_season_periods in ema_mape_values:
    #         writer.writerow(
    #             [material_number, round(best_mapex_one, 2), round(best_mapex_two, 2), round(best_mapex_three, 2),round(best_alpha_d,2),round( best_beta_d,2),round(best_alpha_t,2),round(best_beta_t,2),round(best_gamma_t,2),round(best_season_periods,2)])
    #     plt.show()
    return
if __name__ == '__main__':
    # 物料取值
    data_EMA = range(1,2)
    # 运行
    EMA_run()

LookupError: unknown encoding: ANSI

In [16]:
#croston预测
def Croston_try(n):
    # croston
    def croston(ts, alpha, extra_periods=1):
        # 将输入数据转换为NumPy数组
        d = np.array(ts)
        # 获取历史期间的长度
        cols = len(d)
        # 在需求数组中附加np.nan，以覆盖未来期间
        d = np.append(d, [np.nan] * extra_periods)

        # 初始化水平（a）、周期性（p）和预测（f）
        a, p, f = np.full((3, cols + extra_periods), np.nan)
        q = 1  # 自上次需求观察以来的期数

        # 初始化
        # 找到第一个需求观察值的位置
        first_occurrence = np.argmax(d[:cols] > 0)
        a[0] = d[first_occurrence]
        p[0] = 1 + first_occurrence
        f[0] = a[0] / p[0]

        # 创建所有t+1时刻的预测
        for t in range(0, cols):
            if d[t] > 0:
                a[t + 1] = alpha * d[t] + (1 - alpha) * a[t]
                p[t + 1] = alpha * q + (1 - alpha) * p[t]
                f[t + 1] = a[t + 1] / p[t + 1]
                q = 1
            else:
                a[t + 1] = a[t]
                p[t + 1] = p[t]
                f[t + 1] = f[t]
                q += 1

        # 预测未来期间
        a[cols + 1: cols + extra_periods] = a[cols]
        p[cols + 1: cols + extra_periods] = p[cols]
        f[cols + 1: cols + extra_periods] = f[cols]

        # 创建一个包含需求、预测、周期、水平和误差的Pandas DataFrame
        df_CR = pd.DataFrame.from_dict({"Demand": d, "Forecast": f, "Period": p, "Level": a, "Error": d - f})

        return df_CR

    # croston_sba
    def croston_SBA(ts, alpha, extra_periods=1):
        # 将输入数据转换为NumPy数组
        d = np.array(ts)
        # 获取历史期间的长度
        cols = len(d)
        # 在需求数组中附加np.nan，以覆盖未来期间
        d = np.append(d, [np.nan] * extra_periods)

        # 初始化水平（a）、周期性（p）和预测（f）
        a, p, f = np.full((3, cols + extra_periods), np.nan)
        q = 1  # 自上次需求观察以来的期数

        # 初始化
        # 找到第一个需求观察值的位置
        first_occurrence = np.argmax(d[:cols] > 0)
        a[0] = d[first_occurrence]
        p[0] = 1 + first_occurrence
        f[0] = a[0] / p[0]

        # 创建所有t+1时刻的预测
        for t in range(0, cols):
            if d[t] > 0:
                a[t + 1] = alpha * d[t] + (1 - alpha) * a[t]
                p[t + 1] = alpha * q + (1 - alpha) * p[t]
                f[t + 1] = (1 - alpha / (2)) * (a[t + 1] / p[t + 1])
                q = 1
            else:
                a[t + 1] = a[t]
                p[t + 1] = p[t]
                f[t + 1] = f[t]
                q += 1

        # 预测未来期间
        a[cols + 1: cols + extra_periods] = a[cols]
        p[cols + 1: cols + extra_periods] = p[cols]
        f[cols + 1: cols + extra_periods] = f[cols]

        # 创建一个包含需求、预测、周期、水平和误差的Pandas DataFrame
        df_SBA = pd.DataFrame.from_dict({"Demand": d, "Forecast": f, "Period": p, "Level": a, "Error": d - f})

        return df_SBA

    # croston_tsb
    def croston_tsb(ts, alpha, beta, extra_periods=1):
        d = np.array(ts)  # Transform the input into a numpy array
        cols = len(d)  # Historical period length
        d = np.append(d, [np.nan] * extra_periods)  # Append np.nan into the demand array to cover future periods

        # Level (a), probability (p), and forecast (f)
        a, p, f = np.full((3, cols + extra_periods), np.nan)

        # Initialization
        first_occurrence = np.argmax(d[:cols] > 0)
        a[0] = d[first_occurrence]
        p[0] = 1 / (1 + first_occurrence)
        f[0] = p[0] * a[0]

        # Create all the t+1 forecasts
        for t in range(0, cols):
            if d[t] > 0:
                a[t + 1] = alpha * d[t] + (1 - alpha) * a[t]
                p[t + 1] = beta * (1) + (1 - beta) * p[t]
            else:
                a[t + 1] = a[t]
                p[t + 1] = (1 - beta) * p[t]
            f[t + 1] = p[t + 1] * a[t + 1]

        # Future Forecast
        a[cols + 1: cols + extra_periods] = a[cols]
        p[cols + 1: cols + extra_periods] = p[cols]
        f[cols + 1: cols + extra_periods] = f[cols]

        df_TSB = pd.DataFrame.from_dict({"Demand": d, "Forecast": f, "Period": p, "Level": a, "Error": d - f})
        return df_TSB

    # croston参数迭代
    def cst_optimizer(data):
        best_mape_cst = 999
        # best_mae_cst = 999
        alphas = np.arange(0, 1, 0.1)
        i = 0
        for alpha in alphas:
            i = i + 1
            result_cr = croston(data, alpha)
            forecast_cst = result_cr['Forecast']

            mape_cst = np.mean(np.abs((forecast_cst[:-1].values - data.values) / data.values))
            # mae_cst = mean_absolute_error(data.values, forecast_cst[:-1].values)
            if mape_cst < best_mape_cst:
                best_alpha_cst, best_mape_cst = alpha, mape_cst
            print('cst第', i, '次迭代')
            print("alpha:", round(alpha, 2), "mape:", round(mape_cst, 4))
            # print("alpha:", round(alpha, 2), "mae:", round(mae_cst, 4))
        # 打印最佳的 alpha 值和对应的最佳 MAE 值
        print("best_alpha:", round(best_alpha_cst, 2), "best_mape:", round(best_mape_cst, 4))
        # print("best_alpha:", round(best_alpha_cst, 2), "best_mae:",round(best_mae_cst, 4))
        # 返回最佳的 alpha 值和最佳的 MAE 值
        return best_alpha_cst, best_mape_cst

    # SBA参数迭代
    def sba_optimizer(data):
        best_mape_sba = 999
        # best_mae_sba = 999
        alphas = np.arange(0, 1, 0.1)
        i = 0
        for alpha in alphas:
            i = i + 1
            result_sba = croston_SBA(data, alpha)
            forecast_sba = result_sba['Forecast']

            mape_sba = np.mean(np.abs((forecast_sba[:-1].values - data.values) / data.values))
            # mae_sba = mean_absolute_error(data.values, forecast_sba[:-1].values)

            if mape_sba < best_mape_sba:
                best_alpha_sba, best_mape_sba = alpha, mape_sba
            print('sba第', i, '次迭代')
            print("alpha:", round(alpha, 2), "mape:", round(mape_sba, 4))
            # print("alpha:", round(alpha, 2), "mae:", round(mae_sba, 4))
        # 打印最佳的 alpha 值和对应的最佳 MAE 值
        print("best_alpha:", round(best_alpha_sba, 2), "best_mape:", round(best_mape_sba, 4))
        # print("best_alpha:", round(best_alpha_sba, 2), "best_mae:",round(best_mae_sba, 4))
        # 返回最佳的 alpha 值和最佳的 MAE 值
        return best_alpha_sba, best_mape_sba

    # TSB参数优化
    def tsb_optimizer(data):
        best_mape_tsb = 999
        # best_mae_tsb = 999
        alphas = np.arange(0, 1, 0.1)
        betas = np.arange(0, 1, 0.1)

        i = 0
        for beta in betas:
            for alpha in alphas:
                i = i + 1
                result_tsb = croston_tsb(data, alpha, beta)
                forecast_tsb = result_tsb['Forecast']

                mape_tsb = np.mean(np.abs((forecast_tsb[:-1].values - data.values) / data.values))
                # mae_tsb = mean_absolute_error(data.values, forecast_tsb[:-1].values)
                if mape_tsb < best_mape_tsb:
                    best_alpha_tsb, best_beta_tsb, best_mape_tsb = alpha, beta, mape_tsb
                print('tsb第', i, '次迭代')
                print("alpha:", round(alpha, 2), "beta:", round(beta, 2), "mape:", round(mape_tsb, 4))
            # 打印最佳的 alpha 值和对应的最佳 MAE 值
            print("best_alpha:", round(best_alpha_tsb, 2), "best_beta:", round(best_beta_tsb, 2), "best_mape:",
                  round(best_mape_tsb, 4))
            # print("best_alpha:", round(best_alpha_tsb, 2), "best_beta:", round(best_beta_tsb, 2), "best_mae:",
            #       round(best_mae_tsb, 4))
            # 返回最佳的 alpha 值和最佳的 MAE 值
        return best_alpha_tsb, best_beta_tsb, best_mape_tsb

    # 调用
    df = preprocess_try()
    # df = preprocess()
    data = df.iloc[:, n]

    # 读取列索引名
    col = df.columns

    # 将数据集分为训练集和测试集 65/68= 0.955 预测后面四个
    train_size = int(0.8 * len(data))
    train_data = data.iloc[:train_size]
    test_data = data.iloc[train_size:]

    # 保存最优值
    best_alpha_cst, best_mape_cst = cst_optimizer(data)
    best_alpha_sba, best_mape_sba = sba_optimizer(data)
    best_alpha_tsb, best_beta_tsb, best_mape_tsb = tsb_optimizer(data)
    print('CR输入', best_alpha_cst, best_mape_cst)
    print('SBA输入', best_alpha_sba, best_mape_sba)
    print('TSB输入', best_alpha_tsb, best_beta_tsb, best_mape_tsb)
    # 获取预测值
    result_cr = croston(data, best_alpha_cst)
    result_sba = croston_SBA(data, best_alpha_sba)
    result_tsb = croston_tsb(data, best_alpha_tsb, best_beta_tsb)
    print('CR:', result_cr)
    print('SBA:', result_sba)
    print('TSB:', result_tsb)
    forecast_cst = result_cr['Forecast']
    forecast_sba = result_sba['Forecast']
    forecast_tsb = result_tsb['Forecast']

    # 画图
    plt.figure(figsize=(10, 6))
    plt.plot(df['time'], data, label='实际数据', color='black')
    plt.plot(df['time'], forecast_cst[:-1], label='预测数据—CR', color='g')
    plt.plot(df['time'], forecast_tsb[:-1], label='预测数据—TSB', color='r')
    plt.plot(df['time'], forecast_sba[:-1], label='预测数据—SBA', color='b')
    plt.xlabel('时间')
    plt.ylabel('需求')
    plt.title(
        f'物料号：{col[n]}\n CST MAPE:{round(best_mape_cst, 2)}\n SBA MAPE:{round(best_mape_sba, 2)} \n TSB MAPE:{round(best_mape_tsb, 2)}')
    plt.legend()
    plt.xticks(rotation=45)
    plt.grid(True, linestyle='--', linewidth=0.5)
    plt.tight_layout()
    plt.savefig('cst')
    # plt.show()
    return best_mape_cst, best_mape_sba, best_mape_tsb,col
# crosotn及其变体
def CRO_run():
    # 创建一个空列表，用于存储 MAPE 值
    cro_mape_values = []
    for n in data_CRO:
        # 运行 SMAforecast_try() 并计算 MAPE
        best_mape_cst, best_mape_sba, best_mape_tsb, col = Croston_try(n)
        # 取出对应的物料号
        material_number = col[n]
        print("这是", material_number)
        print( best_mape_cst, best_mape_sba, best_mape_tsb)
        # 将 MAPE 值作为元组添加到列表中
        cro_mape_values.append((material_number, best_mape_cst, best_mape_sba, best_mape_tsb))
    # 将 MAPE 值写入文件
    with open("CRO不平稳=.txt", "w") as f:
        # 循环遍历每个 n 对应的 MAPE 值，并将其写入文件
        for material_number, best_mape_cst, best_mape_sba, best_mape_tsb in cro_mape_values:
            f.write(
                f"物料号：{material_number}, MAPE CRO: {round(best_mape_cst, 2)}, MAPE SBA: {round(best_mape_sba, 2)},MAPE TSB:{round(best_mape_tsb, 2)}\n")
    csv_filename = "CRO不平稳杭宁.csv"
    with open(csv_filename, "a", newline='') as f:  # 使用追加模式，并且指定 newline='' 来避免空行
        writer = csv.writer(f)
        # 循环遍历每个 n 对应的 MAPE 值，并将其写入文件
        for material_number, best_mape_cst, best_mape_sba, best_mape_tsb in cro_mape_values:
            writer.writerow(
                [material_number, round(best_mape_cst, 2), round(best_mape_sba, 2), round(best_mape_tsb, 2)])
    plt.show()
    return
if __name__ == '__main__':
    # 物料取值
    data_CRO = range(1,2)
    # 运行
    CRO_run()

LookupError: unknown encoding: ANSI

In [17]:
#随机森林预测
def forest_try(n):
    # 读取数据
    df = preprocess_try()

    col = df.columns

    # %%
    df = df.iloc[:, [0, n]]
    df.columns = ['time', 'values']

    #
    # 特征工程
    # 解析季度数据为年份和季度
    df['Year'] = df['time'].str.split(' 第').str[0].astype(int)
    df['Quarter'] = df['time'].str.split('第').str[1].str.replace('季度', '').astype(int)

    # 划分数据集
    train_size = int(0.82 * len(df))
    train_data = df.iloc[:train_size]
    test_data = df.iloc[train_size:]

    # 准备特征和目标变量
    X_train = train_data[['Year', 'Quarter']]
    y_train = train_data['values']
    X_test = test_data[['Year', 'Quarter']]
    y_test = test_data['values']

    # print(X_train,y_train)
    # 创建和训练随机森林模型

    # 设置要尝试的参数值列表
    # n_estimators_list = [5,20,40,60,80, 100, 120]
    # max_depth_list = [10,20, 40, 60,80,100]

    n_estimators_list = np.arange(5,100,5)
    max_depth_list = np.arange(5,100,5)

    # 初始化最小 MAPE 和对应的参数组合
    min_mape = float('inf')
    best_n_estimators = None
    best_max_depth = None
    best_predicted = None
    # 循环尝试不同的参数组合
    for n_estimators in n_estimators_list:
        for max_depth in max_depth_list:
            # 创建和训练随机森林模型
            rf_model = RandomForestRegressor(n_estimators=n_estimators,
                                             random_state=0,max_depth=max_depth)
            rf_model.fit(X_train, y_train)
            # 在测试集上评估模型性能
            y_pred = rf_model.predict(X_test)
            mape = np.mean(np.abs((y_pred - y_test) / y_test))

            # 更新最小 MAPE 和对应的参数组合
            if mape < min_mape:
                min_mape = mape
                best_n_estimators = n_estimators
                best_max_depth = max_depth
                best_predicted = y_pred
                # 打印参数组合和性能
            print(f"n_estimators={n_estimators}, max_depth={max_depth}, MAPE={mape},Best_predicted={y_pred}")

    # 打印最佳参数组合和最小的 MAPE
    print(f"Best parameters: n_estimators={best_n_estimators}, max_depth={best_max_depth}, Min MAPE={min_mape},Best_predicted={best_predicted}")

    # # 预测未来3个月的销售额
    # next_months = pd.date_range(start=data.index[-1], periods=3, freq='M')
    # next_months_data = pd.DataFrame({'Month': next_months.month}, index=next_months)
    # predicted_sales = rf_model.predict(next_months_data[['Month']])

    # 打印预测结果
    # print("预测未来3个月的需求：")
    # print(predicted_sales)

    # 计算模型性能
    # y_pred = rf_model.predict(X_test)
    # mape = np.mean(np.abs((y_pred - y_test) / y_test))
    print("MAPE：", min_mape)
    # mae = mean_absolute_error(y_test, y_pred)

    # 将预测结果与真实数据绘制成图表
    # 在绘图前指定字体
    # plt.rcParams["font.family"] = "SimHei"  # 使用宋体字体
    # plt.rcParams["font.size"] = 12  # 设置字体大小
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(df['time'], df['values'], label='真实数量', color='black')
    last_five_time = df['time'].tail(4)
    ax.plot(last_five_time, best_predicted, label='预测数量', color='r')

    # ax.plot(next_months, predicted_sales, label='未来3个月预测', color='g')

    # 其他绘图设置
    plt.xlabel('日期')
    plt.ylabel('数量')
    plt.title(f'物料号：{col[n]} \n MAPE:{round(min_mape, 2)}')

    plt.legend()
    # plt.grid(True)
    plt.xticks(rotation=45)  # 如果需要旋转刻度标签
    plt.tight_layout()  # 确保标签不重叠
    plt.grid(True, linestyle='--', linewidth=0.5)
    plt.tight_layout()
    plt.savefig('FST')
    plt.show()
    return min_mape,col
def FST_run():
    # 创建一个空列表，用于存储 MAPE 值
    fst_mape_values = []
    for n in data_FST:
        # 运行 SMAforecast_try() 并计算 MAPE
        min_mape, col = forest_try(n)
        # 取出对应的物料号
        material_number = col[n]
        print("物料号", material_number)
        print(min_mape)
        # 将 MAPE 值作为元组添加到列表中
        fst_mape_values.append((material_number, min_mape))
    # 将 MAPE 值写入文件
    with open("FST不平稳杭宁.txt", "w") as f:
        # 循环遍历每个 n 对应的 MAPE 值，并将其写入文件
        for material_number, min_mape in fst_mape_values:
            f.write(
                f"物料号：{material_number}, MAPE FSR: {round(min_mape, 2)}\n")
    csv_filename = "FST不平稳杭宁.csv"
    with open(csv_filename, "a", newline='') as f:  # 使用追加模式，并且指定 newline='' 来避免空行
        writer = csv.writer(f)
        # 循环遍历每个 n 对应的 MAPE 值，并将其写入文件
        for material_number,min_mape in fst_mape_values:
            writer.writerow(
                [material_number, round(min_mape, 2)])

    plt.show()
    return
if __name__ == '__main__':
    # 物料取值
    data_FST = range(1,2)
    # 运行
    FST_run()

LookupError: unknown encoding: ANSI

In [ ]:
#ARIMA预测
#基础 ARIMA 模型
def ARIMAforecast_try():
    # 重采样
    def Resampling(data):
        df = data
        # df['time'] = pd.to_datetime(df['time'])
        df.set_index("time", inplace=True)
        train_data = df['2018 第1季度':'2022 第1季度']
        test_data = df['2022 第2季度':'2023 第2季度']
        # 以月为时间间隔取均值,重采样
        # train_data = train_data.resample('M').mean()
        # test_data = test.resample('M').mean()
        # print(train_data, test_data)
        return train_data, test_data

    #### Step 3  差分转平稳
    def stationarity(timeseries):  # 平稳性处理（timeseries 时间序列）
        ## 差分法,保存成新的列
        diff1 = timeseries.diff(1).dropna()  # 1阶差分 dropna() 删除缺失值
        diff2 = diff1.diff(1).dropna()  # 在一阶差分基础上再做一次一阶差分，即二阶查分
        ## 画图
        # diff1.plot(color='red', title='diff 1', figsize=(10, 4))
        # diff2.plot(color='black', title='diff 2', figsize=(10, 4))
        ## 平滑法
        rollmean = timeseries.rolling(window=4, center=False).mean().dropna()  ## 滚动平均
        rollstd = timeseries.rolling(window=4, center=False).std().dropna()  ## 滚动标准差
        ## 画图
        # rollmean.plot(color='yellow', title='Rolling Mean', figsize=(10, 4))
        # rollstd.plot(color='blue', title='Rolling Std', figsize=(10, 4))
        return diff1, diff2, rollmean, rollstd

    # 平稳性检验
    #### Step 4  平稳性检验
    def ADF_test(timeseries):  ## 用于检测序列是否平稳
        column_name = timeseries.columns[0]
        # print(column_name)
        # print(timeseries)
        x = np.array(timeseries[column_name])
        # x = np.array(timeseries['values'])
        adftest = adfuller(x, autolag='AIC')
        print(adftest)
        if adftest[0] < adftest[4]["1%"] and adftest[1] < 10 ** (-8):
            # 对比Adf结果和10%的时的假设检验 以及 P-value是否非常接近0(越小越好)
            print("序列平稳")
            return True
        else:
            print("非平稳序列")
            return False

    # 随机性检验（白噪声检验）
    def random_test(timeseries):
        p_value = acorr_ljungbox(timeseries, lags=1)  # p_value 返回二维数组，第二维为P值
        print(p_value)
        if p_value['lb_pvalue'].iloc[0] < 0.05:
            print("非随机性序列")
            return True
        else:
            print("随机性序列,即白噪声序列")
            return False

    # 利用ACF和PACF判断模型阶数
    def determinate_order_acf(timeseries):
        plot_acf(timeseries, lags=16)  # 延迟数
        plot_pacf(timeseries, lags=7)
        plt.show()

    # ARMA模型
    def ARMA_model(train_data, order):
        arma_model = ARIMA(train_data, order=order)
        arma = arma_model.fit()
        # print(result.summary()) #给出一份模型报告
        # 样本内预测
        in_sample_pred = arma.predict()
        # 样本外预测
        out_sample_pred = arma.predict(start=len(train_data) - 1, end=len(train_data) + 11, dynamic=True)
        # in_sample_pred.plot()
        # train_data.plot()
        return arma, in_sample_pred, out_sample_pred

    # ARIMA模型
    def ARIMA_model(train_data, order):
        arima_model = ARIMA(train_data, order=order)
        arima = arima_model.fit()
        # print(result.summary()) # 给出一份模型报告
        # 样本内预测
        in_sample_pred = arima.predict()
        # 样本外预测
        out_sample_pred = arima.predict(start=len(train_data) - 1, end=len(train_data) + 4, dynamic=True)
        # print(out_sample_pred)
        # print(in_sample_pred)
        return arima, in_sample_pred, out_sample_pred

    # 模型评估
    def evaluate_model(model, train_data, predict_data):
        # （1）利用QQ图检验残差是否满足正态分布
        resid = model.resid  # 求解模型残差
        plt.figure(figsize=(12, 8))
        qqplot(resid, line='q', fit=True)
        plt.xlabel("理论分位数")
        plt.ylabel("样本分位数")
        plt.legend(["残差"], loc="upper left")
        plt.show()
        plt.close()
        # （2）利用D-W检验,检验残差的自相关性
        print('D-W检验值为{}'.format(durbin_watson(resid.values)))
        # （3）利用预测值和真实值的误差检测，这里用的是标准差
        print('标准差为{}'.format(mean_squared_error(train_data, predict_data, sample_weight=None,
                                                 multioutput='uniform_average')))  # 标准差（均方差）

    def draw_picture(row_train_data, out_sample_pred_arima, test_data):
        # print(out_sample_pred)
        # 样本外预测传入 test_data,out_sample_pred
        # 由于预测都是由差分后的平稳序列得出,因此需要对差分后的数据进行还原
        # 还原后绘制同一起点的曲线
        # 将差分后的序列还原,re_out_sample_pred为还原之后
        # re_out_sample_pred_arma = pd.Series(np.array(row_train_data)[-2][0], index=[row_train_data.index[-2]])._append(out_sample_pred_arma[1:]).cumsum()
        re_out_sample_pred_arima = pd.Series(np.array(row_train_data)[-2][0], index=[row_train_data.index[-2]])._append(
            out_sample_pred_arima[1:]).cumsum()

        # # 横坐标
        # x = []
        # start_value = 45  # 设置起始值为44
        # for i in range(24):
        #     x.append(start_value + i)
        # x = np.array(x)
        # x_1 = []
        # for i in range(44):
        #     x_1.append(i + 1)
        # x_1 = np.array(x_1)
        df_all = preprocess_try()
        x = df_all.iloc[:, [0]]
        # print(x)
        # 使用between方法来切片时间列
        # x_train_time = x[x['time'].between('2018 第1季度', '2022 第2季度')]
        # x_test_time = x[x['time'].between('2022 第3季度', '2023 第2季度')]

        # 纵坐标
        y1 = np.array(row_train_data)
        y2 = np.array(test_data)
        # y3 = np.array(re_out_sample_pred_arma[1:])

        y4 = np.array(re_out_sample_pred_arima[1:])
        print(y4)
        mape = np.mean(np.abs((y4 - y2) / y2))

        # 画图
        plt.plot(train_data.index, y1, label='Training Data')
        plt.plot(test_data.index, y2, color='b', label='Test Data')
        # plt.plot(x_test_time, y3, color='r', label='ARMA Prediction')
        print(test_data.index)
        plt.plot(test_data.index, y4, color='r', label='ARIMA Prediction')
        plt.title(f"{df_all.columns[n]},MAPE_ARIMA:{round(mape, 2)}")
        # 添加图例
        plt.legend()
        plt.xticks(rotation=90)
        # plt.show()
        # plt.close()
        return mape

    # 参数

    # n = 6                        # 读取的列索引序号

    # 数据预处理
    df = preprocess_try()
    df = df.iloc[:, [0, n]]  # 选择第0列和第n列，这将保留时间列和第n列

    # # 重采样,划分训练测试
    train_data, test_data = Resampling(df)
    train_data = train_data.astype(float)
    row_train_data = train_data  # 保存差分前的序列,为了后面做评估
    print(row_train_data)
    # 差分转平稳
    Smooth_data = stationarity(train_data)
    print(Smooth_data)

    # for data in zip(Smooth_data, range(4)):             # range(4) 用于判断哪种方法 满足平稳性和白噪声 diff1 diff2 rollmean,rollstd
    #     # print(data[0])
    #     if ADF_test(data[0]) and random_test(data[0]):  # 平稳性和白噪声检测
    #         train_data = data[0]                        # 先用差分，再用平滑,分别对应4个序列
    #         method = data[1]
    #         print(method)                               # 如果是差分做的,那么后面ARIMA模型中要使用这个参数
    #         break
    # print(train_data)

    # determinate_order_acf(train_data)  # ACF定阶

    # 参数
    # order_arma = (1, 0, 1)  # ARMA      p,q其中d=0
    order_arima = (2, 1, 1)  # ARIMA     p,d,q

    # print(order_arma.dtypes)
    # 调用模型
    # arma, in_sample_pred_arma, out_sample_pred_arma = ARMA_model(train_data, order_arma)
    arima, in_sample_pred_arima, out_sample_pred_arima = ARIMA_model(train_data, order_arima)

    # #
    # 模型评价
    # evaluate_model(arma, train_data, in_sample_pred_arma)
    # evaluate_model(arima, train_data, in_sample_pred_arima)
    # #
    # # 可视化
    draw_picture(row_train_data, out_sample_pred_arima, test_data)

#SARIMA模型（考虑季节性）
def ARIMA_try(n):
    # Load the data
    data = preprocess_try()
    col = data.columns
    data = data.iloc[:, [0, n]]
    # A bit of pre-processing to make it nicer
    data.set_index(['time'], inplace=True)

    #
    def convert_to_first_month(quarter_str):
        year, quarter = quarter_str.split(' 第')
        year = int(year)
        quarter = int(quarter[0])

        if quarter == 1:
            month = 1
        elif quarter == 2:
            month = 4
        elif quarter == 3:
            month = 7
        elif quarter == 4:
            month = 10

        return f"{year}-{month}-1"

    #
    # 将data中"time"列的值根据convert_to_first_month函数进行转换
    data.index = data.index.map(convert_to_first_month)
    # 将索引转换为日期时间类型
    data.index = pd.to_datetime(data.index)
    #
    # # 显示更改后的数据
    print(data)
    #
    # 划分训练测试
    train_data = data['2018-4-1':'2022-4-1']
    test_data = data['2022-7-1':'2023-7-1']
    train_data = train_data.astype(float)  # 将数据转换为浮点数类型
    test_data = test_data.astype(float)  # 将数据转换为浮点数类型
    print(test_data)

    # 定义 d 和 q 参数，它们的取值范围在 0 到 1 之间
    q = range(0, 3)
    d = range(0, 2)
    # 要定义 p 参数，其取值范围在 0 到 3 之间
    p = range(0, 4)

    # 生成所有不同的 p、d 和 q 参数的组合。
    pdq = list(itertools.product(p, d, q))

    # 生成所有不同的季节性 p、d 和 q 参数的组合
    seasonal_pdq = [(x[0], x[1], x[2], 4) for x in list(itertools.product(p, d, q))]

    print('Examples of parameter combinations for Seasonal ARIMA...')
    print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
    print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
    print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
    print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))


    AIC = []
    SARIMAX_model = []
    for param in pdq:
        for param_seasonal in seasonal_pdq:
            try:
                mod = sm.tsa.statespace.SARIMAX(train_data,
                                                order=param,
                                                seasonal_order=param_seasonal,
                                                enforce_stationarity=False,
                                                enforce_invertibility=False)

                results = mod.fit()

                print('SARIMAX{}x{} - AIC:{}'.format(param, param_seasonal, results.aic), end='\r')
                AIC.append(results.aic)
                SARIMAX_model.append([param, param_seasonal])
            except:
                continue

    print('The smallest AIC is {} for model SARIMAX{}x{}'.format(min(AIC), SARIMAX_model[AIC.index(min(AIC))][0],
                                                                 SARIMAX_model[AIC.index(min(AIC))][1]))

    # Let's fit this model
    mod = sm.tsa.statespace.SARIMAX(train_data,
                                    order=SARIMAX_model[AIC.index(min(AIC))][0],
                                    seasonal_order=SARIMAX_model[AIC.index(min(AIC))][1],
                                    enforce_stationarity=False,
                                    enforce_invertibility=False)

    results = mod.fit()
    # print('zheshi',results.aic)
    # results.plot_diagnostics(figsize=(20, 14))
    plt.show()

    # 样本内预测——无动态 4
    pred0 = results.get_prediction(start='2021-7-01', dynamic=False)
    pred0_ci = pred0.conf_int()
    data_pred0 = data['2021-07-01':'2022-04-01']

    # 样本内预测——动态 4
    pred1 = results.get_prediction(start='2021-07-01', dynamic=True)
    pred1_ci = pred1.conf_int()
    data_pred1 = data['2021-07-01':'2022-04-01']
    # print('这个hi',data_pred1.index)

    # 样本外预测
    pred2 = results.get_forecast('2023-04-01')
    pred2_ci = pred2.conf_int()
    print(pred2.predicted_mean['2022-07-01':'2023-04-01'])
    # 生成要添加的日期范围
    # new_dates = pd.date_range(start='2023-10-01', end='2024-04-01', freq='3M')
    # 将新日期索引附加到现有索引上
    # data_more = data._append(pd.DataFrame(index=new_dates))
    # 重新排序索引
    # data_more = data_more.sort_index()
    # print(data_more)
    data_pred2 = data['2022-07-01':'2023-04-01']
    # print(data_pred2.index)
    # print( pred2.predicted_mean)

    prediction = pred2.predicted_mean['2022-07-01':'2023-04-01'].values
    # print(prediction)
    # flatten nested list
    truth = list(itertools.chain.from_iterable(test_data.values))
    MAPE = np.mean(np.abs((truth - prediction) / truth))
    # MAE = mean_absolute_error(truth, prediction)
    print('MAPE:', MAPE)
    # print('MAE :', MAE)

    plt.figure(figsize=(10, 6))
    plt.plot(data.index, data, label='原始数据', color='black')
    # plt.plot(data_pred0.index, pred0.predicted_mean, label='无动态样本内预测', color='b')
    # plt.plot(data_pred1.index, pred1.predicted_mean, label='动态样本内预测', color='g')
    plt.plot(data_pred2.index, pred2.predicted_mean, label='样本外预测', color='r')
    plt.ylabel('数量')
    plt.xlabel('日期')
    plt.xticks(rotation=45)  # 如果需要旋转刻度标签
    plt.title(f'物料号：{col[n]} \n MAPE:{round(MAPE, 2)}')
    plt.legend()

    plt.grid(True, linestyle='--', linewidth=0.5)
    plt.tight_layout()
    plt.savefig('SARIMA')
    plt.show()
    return MAPE,col
def ARI_run():
    # 创建一个空列表，用于存储 MAPE 值
    ari_mape_values = []
    for n in data_ARI:
        # 运行 SMAforecast_try() 并计算 MAPE
        MAPE, col = ARIMA_try(n)
        # 取出对应的物料号
        material_number = col[n]
        print("物料号", material_number)
        print(MAPE)
        # 将 MAPE 值作为元组添加到列表中
        ari_mape_values.append((material_number, MAPE))
    # 将 MAPE 值写入文件
    with open("ARI平稳杭宁_5.txt", "w") as f:
        # 循环遍历每个 n 对应的 MAPE 值，并将其写入文件
        for material_number, MAPE in ari_mape_values:
            f.write(
                f"物料号：{material_number}, MAPE ARI: {round(MAPE, 2)}\n")
    csv_filename = "ARI平稳杭宁_5.csv"
    with open(csv_filename, "a", newline='') as f:  # 使用追加模式，并且指定 newline='' 来避免空行
        writer = csv.writer(f)
        # 循环遍历每个 n 对应的 MAPE 值，并将其写入文件
        for material_number,MAPE in ari_mape_values:
            writer.writerow(
                [material_number, round(MAPE, 2)])

    return
if __name__ == '__main__':
    # 物料取值
    data_ARI = range(1,2)
    # 运行
    ARI_run()

In [ ]:
C

In [14]:
#XGBoost预测
import forecast
import warnings, pandas as pd, numpy as np, math  # 告警库 数据处理库  科学计算库
import matplotlib #引入画图数据库
import matplotlib.pyplot as plt
import scipy.fftpack as fftpack
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.seasonal import STL
from statsmodels.tsa.seasonal import seasonal_decompose
plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False
import matplotlib.dates as mdates
# 导入第三方库
import numpy.random as rd  # 导入随机数生成方法
import matplotlib.pyplot as plt  # 数据可视化库
import matplotlib
import xgboost as xgb
import pydot
matplotlib.use('TkAgg')  # 或者使用 'Qt5Agg'
import seaborn as sns  # 高级数据可视化库
from sklearn.model_selection import train_test_split  # 数据集拆分工具
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
import warnings, pandas as pd, numpy as np, math  # 告警库 数据处理库  科学计算库
from keras.models import Sequential  # 导入序贯模型
from keras.layers import Dense  # 导入全连接层
from keras.layers import LSTM  # 导入LSTM层
from keras.utils import plot_model  # 导入 模型绘图工具
import keras.layers as layers  # 导入 层 工具包
import keras.backend as K  # 导入 后端 工具包
import tensorflow as tf  # 导入 TensorFlow 库

from sklearn.metrics import mean_squared_error, explained_variance_score, mean_absolute_error, r2_score  # 模型评估方法

warnings.filterwarnings('ignore')  # 忽略告警

# 定义灰狼优化算法实现函数
def sanitized_gwo(X_train, X_test, y_train, y_test, SearchAgents_no, T, dim, lb, ub):
    Alpha_position = [0, 0]  # 初始化Alpha灰狼的位置
    Beta_position = [0, 0]  # 初始化Beta灰狼的位置
    Delta_position = [0, 0]  # 初始化Delta灰狼的位置

    Alpha_score = float("inf")  # 初始化Alpha灰狼目标函数的值
    Beta_score = float("inf")  # 初始化Beta灰狼目标函数的值
    Delta_score = float("inf")  # 初始化Delta灰狼目标函数的值

    Positions = np.dot(rd.rand(SearchAgents_no, dim), (ub - lb)) + lb  # 初始化第一个搜索位置

    Convergence_curve = np.zeros((1, T))  # 初始化收敛曲线

    iterations = []  # 定义迭代次数列表
    accuracy = []  # 定义准确率列表

    # 迭代求解
    t = 0
    while t < T:  # 循环
        # 迭代每只灰狼位置
        for i in range(0, (Positions.
                shape[0])):
            # 如果搜索位置超出了搜索空间，则需要返回到搜索空间
            for j in range(0, (Positions.shape[1])):
                Flag4ub = Positions[i, j] > ub  # 大于最大值
                Flag4lb = Positions[i, j] < lb  # 小于最小值
                # 如果灰狼的位置在最大值和最小值之间，则不需要调整位置，如果超过最大值，则返回最大值边界；如果低于最小值，则返回最小值边界
                if Flag4ub:  # 判断
                    Positions[i, j] = ub  # 赋值
                if Flag4lb:  # 判断
                    Positions[i, j] = lb  # 赋值

            # 建立XGB模型并训练
            # XGB预测
            model = xgb.XGBRegressor(max_depth=int(abs(Positions[i][0])), learning_rate=int(abs(Positions[i][1]))*0.01, n_estimators=int(abs(Positions[i][2]))*100, random_state=42)
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            score = round(r2_score(y_test, y_pred), 4)  # 计算R2


            # 使错误率降到最低
            fitness_value = 1 - score  # 错误率 赋值 适应度函数值
            if fitness_value < Alpha_score:  # 如果目标函数值小于Alpha灰狼的目标函数值
                Alpha_score = fitness_value  # 然后将Alpha灰狼的目标函数值更新为最优目标函数值
                Alpha_position = Positions[i]  # 同时更新Alpha灰狼的位置到最佳位置
            if fitness_value > Alpha_score and fitness_value < Beta_score:  # 如果目标函数值大于Alpha灰狼的目标函数值并且小于Beta灰狼的目标函数值
                Beta_score = fitness_value  # 然后将Beta灰狼的目标函数值更新为最优目标函数值
                Beta_position = Positions[i]  # 同时更新Beta灰狼的位置到最佳位置
            # 如果目标函数值大于Alpha灰狼的目标函数值并且大于Beta灰狼的目标函数值并且小于Delta灰狼的目标函数值
            if fitness_value > Alpha_score and fitness_value > Beta_score and fitness_value < Delta_score:
                Delta_score = fitness_value  # 然后将Delta灰狼的目标函数值更新为最优目标函数值
                Delta_position = Positions[i]  # 同时更新Delta灰狼的位置到最佳位置

        a = 2 - t * (2 / T)  # 收敛因子从2线性递减到0

        # 循环更新灰狼个体的位置
        for i in range(0, (Positions.shape[0])):
            # 遍历每个维度
            for j in range(0, (Positions.shape[1])):
                # 包围猎物，更新位置
                r1 = rd.random(1)  # 生成0~1之间的随机数
                r2 = rd.random(1)  # 生成0~1之间的随机数
                A1 = 2 * a * r1 - a  # 计算系数向量A
                C1 = 0.5 + (0.5 * math.exp(-j / 500)) + (1.4 * (math.sin(j) / 30))  # 通过时变加速度常数 计算系数向量C

                # alpha灰狼位置更新
                D_alpha = abs(C1 * Alpha_position[j] - Positions[i, j])  # alpha灰狼与其它个体的距离
                X1 = Alpha_position[j] - A1 * D_alpha  # alpha灰狼当前的位置

                r1 = rd.random(1)  # 生成0~1之间的随机数
                r2 = rd.random(1)  # 生成0~1之间的随机数

                A2 = 2 * a * r1 - a  # 计算系数向量A
                C2 = 1 + (1.4 * (1 - math.exp(-j / 500))) + (1.4 * (math.sin(j) / 30))  # 基于差分均值的摄动时变参数 计算系数向量C

                # Beta灰狼位置更新
                D_beta = abs(C2 * Beta_position[j] - Positions[i, j])  # Beta灰狼与其它个体的距离
                X2 = Beta_position[j] - A2 * D_beta  # Beta灰狼当前的位置

                r1 = rd.random(1)  # 生成0~1之间的随机数
                r2 = rd.random(1)  # 生成0~1之间的随机数

                A3 = 2 * a * r1 - a  # 计算系数向量A
                C3 = (1 / (1 + math.exp(-0.0001 * j / T))) + (
                        (0.5 - 2.5) * ((j / T) ** 2))  # 基于sigmoid函数的加速度系数 计算系数向量C

                # Delta灰狼位置更新
                D_delta = abs(C3 * Delta_position[j] - Positions[i, j])  # Delta灰狼与其它个体的距离
                X3 = Delta_position[j] - A3 * D_delta  # Delta灰狼当前的位置

                # 位置更新
                Positions[i, j] = (X1 + X2 + X3) / 3

        t = t + 1
        iterations.append(t)  # 迭代次数存入列表中
        accuracy.append(abs((100 - Alpha_score) / 100))  # 计算准确率
        print('----------------迭代次数----------------' + str(t))

    best_depth = Alpha_position[0]
    best_lr = Alpha_position[1]  # 最优位置  即最优参数值
    best_estimators = Alpha_position[2]  # 最优位置  即最优参数值

    return best_depth, best_lr, best_estimators, iterations, accuracy  # 返回数据

if __name__ == '__main__':

    df= Forecast.preprocess_try()
    col = df.columns
    n=16
    # n=7
    # %%
    df = df.iloc[:, [0, n]]
    df.columns = ['time', 'values']

    #
    # 特征工程
    # 解析季度数据为年份和季度
    df['Year'] = df['time'].str.split(' 第').str[0].astype(int)
    df['Quarter'] = df['time'].str.split('第').str[1].str.replace('季度', '').astype(int)

    # 划分数据集
    train_size = int(0.82 * len(df))
    train_data = df.iloc[:train_size]
    test_data = df.iloc[train_size:]

    # 准备特征和目标变量
    X_train = train_data[['Year', 'Quarter']]
    y_train = train_data['values']
    X_test = test_data[['Year', 'Quarter']]
    y_test = test_data['values']

    SearchAgents_no = 50  # 灰狼个数
    T = 2  # 最大迭代次数
    dim = 3  # 维度 需要优化两个变量 - depth lr es
    lb = 1  # 最小值限制
    ub = 50  # 最大值限制

    print('----------------3.调用灰狼算法函数-----------------')
    best_depth, best_lr, best_estimators, iterations, accuracy = sanitized_gwo(X_train, X_test, y_train, y_test,
                                                                               SearchAgents_no, T,
                                                                               dim,
                                                                               lb, ub)

    print('----------------4. 最优结果展示-----------------')
    print("The best depth is " + str(int(abs(best_depth))))  # 输出数据
    print("The best lr is " + str(int(abs(best_lr)) * 0.01))  # 输出数据
    print("The best estimators is " + str(int(abs(best_estimators)) * 100))  # 输出数据

    print('----------------5. 应用优化后的最优参数值构建SVR回归模型-----------------')
    # 建立XGB模型并训练
    # XGB预测
    model = xgb.XGBRegressor(max_depth=int(abs(best_depth)), learning_rate=int(abs(best_lr)) * 0.01,
                             n_estimators=int(abs(best_estimators)) * 100, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mape = np.mean(np.abs((y_pred - y_test) / y_test))
    print('----------------模型评估-----------------')
    # 模型评估
    print('**************************输出测试集的模型评估指标结果*******************************')
    print('XGB回归模型-最优参数-MAPE：', mape)


    # 真实值与预测值比对图
    plt.figure(figsize=(12, 6))
    plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
    plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号
    plt.plot(df['time'], df['values'], label='真实数量', color='black')
    last_five_time = df['time'].tail(4)
    # plt.plot(last_five_time, y_test, label='预测数量', color='r')
    plt.plot(last_five_time, y_pred, label='预测数量', color='r')
    # plt.plot(range(len(y_test)), y_test, color="blue", linewidth=1.5, linestyle="-")  # 绘图
    # plt.plot(range(len(y_pred)), y_pred, color="red", linewidth=1.5, linestyle="-.")  # 绘图
    plt.legend(['真实值', '预测值'])  # 设置图例
    plt.title("智能灰狼优化算法优化XGB回归模型真实值与预测值比对图")  # 设置标题名称
    plt.show()  # 显示图片

XGBoostError: 
XGBoost Library (libxgboost.dylib) could not be loaded.
Likely causes:
  * OpenMP runtime is not installed
    - vcomp140.dll or libgomp-1.dll for Windows
    - libomp.dylib for Mac OSX
    - libgomp.so for Linux and other UNIX-like OSes
    Mac OSX users: Run `brew install libomp` to install OpenMP runtime.

  * You are running 32-bit Python on a 64-bit OS

Error message(s): ["dlopen(/Users/bulingzheng/Library/Python/3.9/lib/python/site-packages/xgboost/lib/libxgboost.dylib, 0x0006): Library not loaded: @rpath/libomp.dylib\n  Referenced from: <89AD948E-E564-3266-867D-7AF89D6488F0> /Users/bulingzheng/Library/Python/3.9/lib/python/site-packages/xgboost/lib/libxgboost.dylib\n  Reason: tried: '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/libomp/lib/libo